# Neural Network for classification of sex based on facial feature vectors
Feature vectors obtained from FaceNet. Face detection with RetinaFace (insightface)

Neural network with ad-hoc architecture (5 fully connected layers)

Faces from Fairfaces train dataset

In [1]:
import random
import torch
from torch import nn, optim
import math
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print("Loading templates from gallery file.")
with open("/media/rafael/Windows-SSD/recfac/fairface_train_facenet.gal","rb") as f:
    gallery = pickle.load(f, encoding="latin1")
print("{} templates loaded from gallery file.".format(len(gallery)))

Loading templates from gallery file.
86744 templates loaded from gallery file.


In [6]:
labels_dict = pd.read_csv('/media/rafael/Windows-SSD/recfac/bases/fairfaces/fairface_label_train.csv', header=0, index_col=0, usecols=[0,2] ).to_dict()['gender']
labels_pd = pd.DataFrame.from_dict(labels_dict, orient='index', columns=['gender'])
features_pd = pd.DataFrame.from_dict(gallery, orient='index', columns=['status','features'])
bads = features_pd[features_pd['status'] == 'no face']
keys = ['train/' + index for index in bads.index.tolist()]
labels_pd.drop(index=keys, inplace=True)
keys = [index for index in bads.index.tolist()]
features_pd.drop(index=keys, inplace=True)
len(features_pd), len(labels_pd)

(86664, 86664)

In [7]:
labels_pd.sort_index(axis=0, level=None, ascending=True, inplace=True, kind='quicksort')
features_pd.sort_index(axis=0, level=None, ascending=True, inplace=True, kind='quicksort')

In [8]:
features_pd.head()

,status,features
1.jpg,ok,"[0.22564129531383514, -0.5401293635368347, -0...."
10.jpg,ok,"[-0.7215465307235718, 0.26587170362472534, -0...."
100.jpg,ok,"[1.1927777528762817, 0.10555720329284668, 0.32..."
1000.jpg,ok,"[0.8545346260070801, 1.7964006662368774, -2.07..."
10000.jpg,ok,"[-0.0444677472114563, -0.2601420283317566, -0...."


In [9]:
X = np.array(features_pd['features'])
y = np.array(labels_pd['gender'])

In [10]:
X = np.column_stack(list(zip(*X)))

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y_enc =le.transform(y)
le.classes_

array(['Female', 'Male'], dtype=object)

In [12]:
X = torch.tensor(X).float().to(device)
y = torch.tensor(y_enc).long().to(device)

In [13]:
tuple(X.size()),tuple(y.size())

((86664, 128), (86664,))

In [14]:
y

tensor([1, 1, 0,  ..., 1, 1, 0], device='cuda:0')

In [15]:
X

tensor([[ 0.2256, -0.5401, -0.5190,  ..., -1.0052,  0.3861, -0.0374],
        [-0.7215,  0.2659, -0.4371,  ..., -0.9571,  1.1012,  0.3123],
        [ 1.1928,  0.1056,  0.3253,  ..., -0.0420, -0.6737, -1.5649],
        ...,
        [ 0.8388, -0.1194, -0.2304,  ..., -2.6394,  1.7664, -0.2935],
        [-0.4859, -0.6224, -1.3966,  ..., -1.2480,  0.1008,  1.6479],
        [-0.2739,  0.1696, -1.8366,  ..., -0.6585,  0.0236, -1.0443]],
       device='cuda:0')

In [16]:
D = X.shape[1]
C = 2
H = 200

In [78]:
lr = 1e-3
lambda_l2 = 1e-3

In [79]:
def train(X, y, n_epochs, batch_size):
    global model
    # nn package to create our linear model
    # each Linear module has a weight and bias
    model = nn.Sequential(
        nn.Linear(D, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, C)
    )
    model.to(device)
    # nn package also has different loss functions.
    # we use cross entropy loss for our classification task
    criterion = torch.nn.CrossEntropyLoss()

    # we use the optim package to apply
    # ADAM for our parameter updates
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lambda_l2) # built-in L2
    
    # Training
    for epoch in range(n_epochs):

        # X is a torch Variable
        permutation = torch.randperm(X.size()[0])

        for i in range(0,X.size()[0], batch_size):
            optimizer.zero_grad()

            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X[indices], y[indices]
            
            y_pred = model.forward(batch_x.to(device))
            loss = criterion(y_pred,batch_y)
            
            score, predicted = torch.max(y_pred, 1)
            acc = 100*(batch_y == predicted).sum().float() / len(batch_y)
            
            print("Epoch: {} | Batch: {}: | Loss: {:.4f} | Accuracy: {:.2f}".format(epoch, i//batch_size+1, loss, acc),end='\r')

            loss.backward()
            optimizer.step()
        print()
    
    
"""for t in range(epochs):
    
        # Feed forward to get the logits
        y_pred = model(X)
        
        # Compute the loss and accuracy
        loss = criterion(y_pred, y)
        score, predicted = torch.max(y_pred, 1)
        acc = (y == predicted).sum().float() / len(y)
        print("Training - [EPOCH]: {}, [LOSS]: {:.6f}, [ACCURACY]: {:.3f}".format(t+1, loss.item(), acc),end='\r')
        #display.clear_output(wait=True)
        
        # zero the gradients before running
        # the backward pass.
        optimizer.zero_grad()
        
        # Backward pass to compute the gradient
        # of loss w.r.t our learnable params. 
        loss.backward()
        
        # Update params
        optimizer.step()"""

'for t in range(epochs):\n    \n        # Feed forward to get the logits\n        y_pred = model(X)\n        \n        # Compute the loss and accuracy\n        loss = criterion(y_pred, y)\n        score, predicted = torch.max(y_pred, 1)\n        acc = (y == predicted).sum().float() / len(y)\n        print("Training - [EPOCH]: {}, [LOSS]: {:.6f}, [ACCURACY]: {:.3f}".format(t+1, loss.item(), acc),end=\'\r\')\n        #display.clear_output(wait=True)\n        \n        # zero the gradients before running\n        # the backward pass.\n        optimizer.zero_grad()\n        \n        # Backward pass to compute the gradient\n        # of loss w.r.t our learnable params. \n        loss.backward()\n        \n        # Update params\n        optimizer.step()'

In [80]:
def cross_val(X, y, n):
    acc=[]
    err = []
    print('K-fold cross Validation - {} folds.'.format(n))
    for i in range(0,n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1./n)
        train(X_train, y_train, 10, 1024)
        y_pred = model(X_test.to(device))
        score, predicted = torch.max(y_pred, 1) # predicted = class predicted, one-hot encoded
        errors = (y_test != predicted).sum()
        accuracy = 100*(1-float(errors)/len(y_test))
        print("Validation - Errors - fold {}: {}/{}   -   Accuracy: {:.2f}%".format(i+1,errors,len(y_test),accuracy))
        acc.append(accuracy)
        err.append(float(errors))
    acc = np.array(acc)
    err = np.array(err)
    return acc, err

In [81]:
n = 5 # number of folds
accuracy, errors = cross_val(X,y,n)
print('Mean accuracy: {:.2f}%\nMean errors/total: {}/{}'.format(accuracy.mean(),errors.mean(),len(y)//n)) 

K-fold cross Validation - 5 folds.
Epoch: 0 | Batch: 68: | Loss: 0.3000 | Accuracy: 86.58
Epoch: 1 | Batch: 68: | Loss: 0.2750 | Accuracy: 86.45
Epoch: 2 | Batch: 68: | Loss: 0.2631 | Accuracy: 88.24
Epoch: 3 | Batch: 68: | Loss: 0.2853 | Accuracy: 86.72
Epoch: 4 | Batch: 68: | Loss: 0.2463 | Accuracy: 88.38
Epoch: 5 | Batch: 68: | Loss: 0.2169 | Accuracy: 91.70
Epoch: 6 | Batch: 68: | Loss: 0.2439 | Accuracy: 89.49
Epoch: 7 | Batch: 68: | Loss: 0.2354 | Accuracy: 88.38
Epoch: 8 | Batch: 68: | Loss: 0.2542 | Accuracy: 88.52
Epoch: 9 | Batch: 68: | Loss: 0.2153 | Accuracy: 89.90
Validation - Errors - fold 1: 2068/17333   -   Accuracy: 88.07%
Epoch: 0 | Batch: 68: | Loss: 0.2764 | Accuracy: 87.55
Epoch: 1 | Batch: 68: | Loss: 0.2867 | Accuracy: 86.72
Epoch: 2 | Batch: 68: | Loss: 0.2805 | Accuracy: 86.86
Epoch: 3 | Batch: 68: | Loss: 0.2488 | Accuracy: 88.24
Epoch: 4 | Batch: 68: | Loss: 0.2740 | Accuracy: 87.69
Epoch: 5 | Batch: 68: | Loss: 0.2274 | Accuracy: 90.18
Epoch: 6 | Batch: 68:

In [82]:
train(X, y, 10, 1024)

Epoch: 0 | Batch: 85: | Loss: 0.3042 | Accuracy: 85.03
Epoch: 1 | Batch: 85: | Loss: 0.2642 | Accuracy: 87.04
Epoch: 2 | Batch: 85: | Loss: 0.2509 | Accuracy: 88.12
Epoch: 3 | Batch: 85: | Loss: 0.2604 | Accuracy: 86.11
Epoch: 4 | Batch: 85: | Loss: 0.2791 | Accuracy: 87.04
Epoch: 5 | Batch: 85: | Loss: 0.2264 | Accuracy: 89.20
Epoch: 6 | Batch: 85: | Loss: 0.2270 | Accuracy: 89.35
Epoch: 7 | Batch: 85: | Loss: 0.2037 | Accuracy: 90.90
Epoch: 8 | Batch: 85: | Loss: 0.2203 | Accuracy: 90.90
Epoch: 9 | Batch: 85: | Loss: 0.2291 | Accuracy: 89.66


In [83]:
torch.save(model.state_dict(), '/media/rafael/Windows-SSD/recfac/bases/fairfaces/sex_from_features_train_facenet.pth')